# Cleaning Data in SQL
Data from your queries may often contain missing values, duplicate rows, or data in the wrong format. It is crucial to be able to clean the data you retrieve through your queries. This template runs through the basics of handling missing data, duplicate rows, and string cleaning.

This template connects to a database containing different course datasets by default. However, you can use any integration you want, so long as you have access to a table with text data. You can learn more about integrations [here](https://workspace-docs.datacamp.com/integrations/what-is-an-integration).

💡&nbsp;&nbsp;_The data we will use is contained various schemas of the Course Databases database. To edit the queries, you must add the schema name when referencing tables in this database (e.g., `world.currencies`)._

## Missing values
Missing or null values are common in data and can often present a problem for future analyses. You can inspect the number of missing values in a column by filtering the data for null values and performing an aggregation. 

In the example below, we filter for records where the `unemployment_rate` is `NULL` and then `COUNT()` the number of rows.

In [ ]:
SELECT COUNT(*) AS number_missing_unemployment_rates
FROM world.economies
WHERE unemployment_rate IS NULL

### Filling missing values
Once you have identified the missing values, you can handle them in several ways. 

Here, we use `COALESCE()` to replace NULL values with the average unemployment rate (which is accessed via a subquery). `COALESCE()` works by returning the first argument if it is not null. If it is null, it returns the second argument, and so forth. If the `unemployment_rate` column is `NULL`, it returns the second argument, which is the average unemployment we calculate with a subquery.

In [ ]:
SELECT
	code,
    unemployment_rate,
	COALESCE(unemployment_rate, 
             (SELECT AVG(unemployment_rate) FROM world.economies)) AS filled_unemployment_rate
FROM world.economies

## Duplicate rows
Another data issue you may come across is duplicate rows. To identify them, you can use `ROW_NUMBER()` to assign numbers to rows based on identical combinations. By choosing the `PARTITION` of the window function, you can specify over which columns you want to look for duplicates.

In the example below, we use `PARTITION BY` to assign row numbers based on the combination of country code and unemployment rate. As you can see from the query results, duplicate rows have a value of 2 or greater.

In [ ]:
SELECT *
FROM (
    SELECT 
        code, 
        unemployment_rate,
        ROW_NUMBER() OVER(PARTITION BY code, unemployment_rate) AS row_number
    FROM world.economies
) AS sub

### Discarding duplicate rows
Removing duplicate rows is just as simple as identifying them. To do so, you simply need to change your filter to select `row_number`s with a value of 1.

In [ ]:
SELECT *
FROM (
    SELECT 
        code, 
        unemployment_rate,
        ROW_NUMBER() OVER(PARTITION BY code, unemployment_rate) AS row_number
    FROM world.economies
) AS sub
WHERE row_number = 1

## Invalid data
If you are aware of invalid data, you can find (and remove) it using pattern-matching.

In the example below, we search for rows where the `indep_year` contains a negative value. To do so, we convert the column to text using `::TEXT`, and then use `LIKE` and our pattern. The pattern we use searches for a minus sign (`-`), followed by any other characters (using the wildcard `%`).

In [ ]:
SELECT indep_year
FROM world.countries
WHERE indep_year::TEXT LIKE '-%'

You can also use pattern matching to find rows with similar variants. In the example below, we use a pattern to identify all rows with `Monarchy` in the `gov_form` column.

Our pattern searches for any row with "Monarchy". We use the `%` wildcard characters to allow for words/whitespace on either side of the word we are searching for.

In [ ]:
SELECT DISTINCT name, gov_form
FROM world.countries
WHERE gov_form LIKE '%Monarchy%'

### Fixing invalid data
There are a variety of ways you can fix invalid data. One way is to use a `CASE` statement to recategorize the data. In the example below, we convert all `gov_form` rows that contain "Monarchy" to "Monarchy". The remaining entries are left as they are.

In [ ]:
SELECT DISTINCT 
	name, 
    gov_form,
    CASE WHEN gov_form LIKE '%Monarchy%' THEN 'Monarchy' 
    ELSE gov_form END AS fixed_gov_form
FROM world.countries
WHERE gov_form LIKE '%Monarchy%'

## Data types
You can query the `columns` table of the `information_schema` database to learn more about the data types of the table you will be working with.

In the query below, we retrieve each column and the data type for the `rental` table in the `dvdrentals` schema.

In [ ]:
SELECT 
	column_name,
    data_type
FROM information_schema.columns
WHERE table_name = 'rental'

### Converting data types
Sometimes, a column may not be the correct data type. You can use `CAST()` or `::` to convert a column to a specified type.

In the example below, we convert two strings and two integers to different data types. The latter two columns produce identical results. The column `integer_to_text` converts the integer 16 to text using `CAST()`. The column `integer_to_text_with_operator` does the same with the cast operator `::`.

In [ ]:
SELECT
	CAST('42' AS INTEGER) AS string_to_integer,
    CAST('2022-06-01' AS DATE) AS string_to_date,
    CAST(16 AS TEXT) AS integer_to_text,
    16::TEXT AS integer_to_text_with_operator

### Converting date formats
Sometimes you may want to convert a date into a new format. You can use `TO_CHAR()` to convert a given date to a provided format.

In the example below, we use the short name of the month and the last two digits of the year to convert the precise rental date to a month_year column.

_Note: In Workspace, SQL queries are converted to pandas DataFrames. As a result, some formatting strings may result in Python automatically interpreting the result as a datetime and converting the date back to the original format._

In [ ]:
SELECT 
	rental_id, 
    rental_date, 
    TO_CHAR(rental_date, 'Mon-YY') AS month_year
FROM dvdrentals.rental
LIMIT 5

## Next steps
Want to learn more techniques for cleaning data in PostgreSQL? Check out [Cleaning Data in PostgreSQL Databases](https://app.datacamp.com/learn/courses/cleaning-data-in-postgresql-databases)!

Alternatively, if you are interested in applying these skills to other SQL databases, check out our [sample integrations](https://app.datacamp.com/workspace/datasets?selectedLabels=%5B%22sql%22%5D) here!